In [2]:

import Bio
import numpy as np
from Bio import SeqIO
import sys
import csv
import os
import random as random
from functions_collection_copy import *
import time as time

score_matrix={'a': {'a': 0, 'c': 1, 'g': 1, 't': 1}, 'c': {'a': 1, 'c': 0, 'g': 1, 't': 1}, 'g': {'a': 1, 'c': 1, 'g': 0, 't': 1}, 't': {'a': 1, 'c': 1, 'g': 1, 't': 0}}
gap_cost=1


In [18]:
families=4
str_in_fams=3
rep=1
len_str=100
internal_var=[0.1,0.25,0.35,0.5,0.75,0.85,1]
external_var=[0.1,0.25,0.35,0.5,0.75,0.85,1]

random_sequences_collection = []

csv_file_path="testing_outputs/worst_case_vs_actual_new_distance_k12_c4.csv"
with open(csv_file_path, 'a', newline='') as csvfile:
                            csvwriter = csv.writer(csvfile)
                            csvwriter.writerow(["c","k","int_var","ex_var","type","ratio"])
for i in range(rep):
    for int_var in internal_var:
        for ex_var in external_var:
            strings_in_families=make_strings_in_families(families,str_in_fams,len_str,int_var,ex_var)
            matrix_krus,MSA_list_krus, total_cost_krus,in_which_MSA_is_it_krus,who_aligned_to_who_krus=new_assembly_gradual_x(strings_in_families,score_matrix,gap_cost)
            matrix_gus,MSA_list_gus, total_cost_gus,in_which_MSA_is_it_gus,middle_string_gus=new_assembly_Gus_x(strings_in_families,score_matrix,gap_cost)
            #calculate internal var
            print(matrix_krus)

            print(who_aligned_to_who_krus)
            nr_strs=str_in_fams*families
            who_aligned_to_who_gus={}
            who_aligned_to_who_gus = {i: int(middle_string_gus) for i in range(0, nr_strs) if i != int(middle_string_gus)}
            print(who_aligned_to_who_gus)

            distance_matrix=matrix_krus
            # Define the number of elements in each group
            elements_in_each_group = str_in_fams

            # Calculate the number of groups
            num_groups = len(distance_matrix) // elements_in_each_group

            # Define group indices
            group_indices = [list(range(i * elements_in_each_group, (i + 1) * elements_in_each_group)) for i in range(num_groups)]

            # Calculate average distance within each group
            average_distance_within_groups = []
            for group in group_indices:
                distances_within_group = [distance_matrix[i, j] for i in group for j in group if i != j]
                average_distance_within_group = np.mean(distances_within_group)
                average_distance_within_groups.append(average_distance_within_group)

            # Calculate average distance between different groups
            average_distance_between_groups = []
            for i in range(num_groups):
                for j in range(i + 1, num_groups):
                    group1 = group_indices[i]
                    group2 = group_indices[j]
                    distances_between_groups = [distance_matrix[i, j] for i in group1 for j in group2]
                    average_distance_between_group = np.mean(distances_between_groups)
                    average_distance_between_groups.append(average_distance_between_group)

            average_average_witin_group=np.mean(average_distance_within_group)
            average_average_between_groups=np.mean(average_distance_between_groups)
            # Print results

            worst_case_gus=(average_average_witin_group)*((families*str_in_fams)/families-1)*((families*str_in_fams)-1)+(average_average_between_groups)*((families*str_in_fams)-((families*str_in_fams)/families))*((families*str_in_fams)-1)
            #worst_case_krus=internal_var*((families*str_in_fams)-families)*((families*str_in_fams)-1)+external_var*(families-1)((families*str_in_fams)*(((families*str_in_fams)-(families*str_in_fams)*(1/families))/families))
            worst_case_krus = (average_average_witin_group)*((families * str_in_fams) - families) * ((families * str_in_fams) - 1) + (average_average_between_groups) * (families - 1) * ((families * str_in_fams) * (((families * str_in_fams) - (families * str_in_fams) * (1 / families)) / families))
            print(matrix_krus)
            print(worst_case_gus)
            print(worst_case_krus)
            print(total_cost_gus)
            print(total_cost_krus)
            gus_ratio=total_cost_gus/worst_case_gus
            krus_ratio=total_cost_krus/worst_case_krus

            with open(csv_file_path, 'a', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow([families,(families*str_in_fams),int_var,ex_var,"kruskal",krus_ratio])
                csvwriter.writerow([families,(families*str_in_fams),int_var,ex_var,"gusfield",gus_ratio])


































[[ 0.  9.  7.  8. 14. 16.  8. 12. 15.  9. 18. 20.]
 [ 9.  0. 16. 15. 20. 22. 17. 21. 22. 17. 26. 26.]
 [ 7. 16.  0. 15. 21. 22. 14. 18. 21. 15. 24. 22.]
 [ 8. 15. 15.  0.  6.  8. 16. 18. 22. 16. 23. 26.]
 [14. 20. 21.  6.  0. 14. 21. 21. 26. 18. 25. 30.]
 [16. 22. 22.  8. 14.  0. 23. 23. 28. 23. 30. 29.]
 [ 8. 17. 14. 16. 21. 23.  0.  6.  8. 17. 26. 26.]
 [12. 21. 18. 18. 21. 23.  6.  0. 14. 20. 29. 27.]
 [15. 22. 21. 22. 26. 28.  8. 14.  0. 23. 31. 31.]
 [ 9. 17. 15. 16. 18. 23. 17. 20. 23.  0.  9. 13.]
 [18. 26. 24. 23. 25. 30. 26. 29. 31.  9.  0. 22.]
 [20. 26. 22. 26. 30. 29. 26. 27. 31. 13. 22.  0.]]
[['3', '4'], ['6', '7'], ['0', '2'], ['0', '3'], ['0', '6'], ['3', '5'], ['6', '8'], ['0', '1'], ['0', '9'], ['9', '10'], ['9', '11']]
{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0}
[[ 0.  9.  7.  8. 14. 16.  8. 12. 15.  9. 18. 20.]
 [ 9.  0. 16. 15. 20. 22. 17. 21. 22. 17. 26. 26.]
 [ 7. 16.  0. 15. 21. 22. 14. 18. 21. 15. 24. 22

In [7]:
import numpy as np

distance_matrix=matrix_krus

# Define the number of elements in each group
elements_in_each_group = 3

# Calculate the number of groups
num_groups = len(distance_matrix) // elements_in_each_group

# Define group indices
group_indices = [list(range(i * elements_in_each_group, (i + 1) * elements_in_each_group)) for i in range(num_groups)]

# Calculate average distance within each group
average_distance_within_groups = []
for group in group_indices:
    distances_within_group = [distance_matrix[i, j] for i in group for j in group if i != j]
    average_distance_within_group = np.mean(distances_within_group)
    average_distance_within_groups.append(average_distance_within_group)

# Calculate average distance between different groups
average_distance_between_groups = []
for i in range(num_groups):
    for j in range(i + 1, num_groups):
        group1 = group_indices[i]
        group2 = group_indices[j]
        distances_between_groups = [distance_matrix[i, j] for i in group1 for j in group2]
        average_distance_between_group = np.mean(distances_between_groups)
        average_distance_between_groups.append(average_distance_between_group)

average_average_witin_group=np.mean(average_distance_within_group)
average_average_between_groups=np.mean(average_distance_between_groups)
# Print results
print("Average distance within each group:", average_average_witin_group)
print("Average distance between different groups:", average_average_between_groups)


Average distance within each group: 8.333333333333334
Average distance between different groups: 26.40740740740741
